In [1]:
import os
import shutil
import yaml
from tqdm import tqdm

In [ ]:
path_original = '../datasets/rocole_seg'
path_novo = '../datasets/rocole_unificado'

#mapa de conversão
mapa_classes = {
    0: 0,  # healthy -> healthy
    1: 1,  # mite -> mite
    2: 2,  # rust_1 -> rust
    3: 2,  # rust_2 -> rust
    4: 2,  # rust_3 -> rust
    5: 2   # rust_4 -> rust
}

classes = ['healthy', 'red_spider_mite', 'rust']


def processar_dataset():
    # 1. Copiar o dataset
    if os.path.exists(path_novo):
        shutil.rmtree(path_novo)
    print(f" copiando dataset para {path_novo}")
    shutil.copytree(path_original, path_novo)
    
    # 2. Listar arquivos
    print("reescrevendo rótulos")
    arquivos_txt = []
    for root, dirs, files in os.walk(path_novo):
        for file in files:
            if file.endswith('.txt'):
                arquivos_txt.append(os.path.join(root, file))

    # 3. Ler e substituir
    for txt_path in tqdm(arquivos_txt):
        novas_linhas = []
        with open(txt_path, 'r') as f:
            linhas = f.readlines()
            
        for linha in linhas:
            partes = linha.strip().split()
            
            # e a linha estiver vazia, pula
            if not partes: continue
            
            # Tenta converter o primeiro item para número.
            try:
                classe_antiga = int(partes[0])
            except ValueError:
                continue 

            coords = partes[1:]
            
            if classe_antiga in mapa_classes:
                nova_classe = mapa_classes[classe_antiga]
                nova_linha = f"{nova_classe} " + " ".join(coords) + "\n"
                novas_linhas.append(nova_linha)
        
        with open(txt_path, 'w') as f:
            f.writelines(novas_linhas)

    #  atualizar YAML
    yaml_path = os.path.join(path_novo, 'data.yaml')
    # Verifica se o arquivo existe antes de abrir
    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)
        
        data['nc'] = len(classes)
        data['names'] = classes
        data['path'] = os.path.abspath(path_novo)
        data['train'] = 'train/images'
        data['val'] = 'valid/images'
        if 'test' in data: data['test'] = 'test/images'

        with open(yaml_path, 'w') as f:
            yaml.dump(data, f)
        
        print("\n novo dataset criado.")
        print(f"Novas Classes: {classes}")
    else:
        print("\n Aviso: Arquivo data.yaml não encontrado na raiz da cópia.")

if __name__ == "__main__":
    processar_dataset()

📦 Copiando dataset para ../datasets/rocole_unificado...
🔄 Reescrevendo rótulos...


100%|██████████| 1492/1492 [00:08<00:00, 178.70it/s]


✅ Sucesso! Dataset unificado criado.
Novas Classes: ['healthy', 'red_spider_mite', 'rust']
